In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
%matplotlib inline

In [4]:
def show_img(img,name='img', hstack=False):
    if not hstack:
        for i in range(len(img)):
            cv2.imshow(name + str(i), img[i])
    else:
        res = np.hstack(img)
        cv2.imshow("res", res)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
file_path = "../img/rate_test.png"

In [3]:
img = cv2.imread(file_path, -1)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [9]:
contours,hierarchy = cv2.findContours(gray, 2, 2)

for cnt in contours:

    # 最小外界矩形的宽度和高度
    width, height = cv2.minAreaRect(cnt)[1]
    print(width, height)

    if width* height > 100:
        # 最小的外接矩形
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)  # 获取最小外接矩形的4个顶点
        box = np.int0(box)
        print(box)

        if 0 not in box.ravel():
            print("绘制最小外界矩形")

            '''绘制最小外界矩形
            for i in range(4):
                cv2.line(image, tuple(box[i]), tuple(box[(i+1)%4]), 0)  # 5
            '''
            # 旋转角度
            theta = cv2.minAreaRect(cnt)[2]
            if abs(theta) <= 45:
                print('图片的旋转角度为%s.'%theta)
                angle = theta

# 仿射变换,对图片旋转angle角度
h, w, c = img.shape
center = (w//2, h//2)
angle = 0
M = cv2.getRotationMatrix2D(center, angle, 1.0)
rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

911.0 1279.0
[[1279  911]
 [   0  911]
 [   0    0]
 [1279    0]]
绘制最小外界矩形


In [11]:
cv2.imshow('ret1', img)
cv2.imshow('ret2', box)
cv2.waitKey(0)

error: OpenCV(4.2.0) C:/projects/opencv-python/opencv/modules/highgui/src/precomp.hpp:137: error: (-215:Assertion failed) src_depth != CV_16F && src_depth != CV_32S in function 'convertToShow'


In [35]:
def Img_Outline(input_dir):
    original_img = cv2.imread(input_dir)
    gray_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray_img, (9, 9), 0)                     # 高斯模糊去噪（设定卷积核大小影响效果）
    _, RedThresh = cv2.threshold(blurred, 165, 255, cv2.THRESH_BINARY)  # 设定阈值165（阈值影响开闭运算效果）
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))          # 定义矩形结构元素
    closed = cv2.morphologyEx(RedThresh, cv2.MORPH_CLOSE, kernel)       # 闭运算（链接块）
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)           # 开运算（去噪点）
    return original_img, gray_img, RedThresh, closed, opened


def findContours_img(original_img, opened):
    contours, hierarchy = cv2.findContours(opened, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)
    c = sorted(contours, key=cv2.contourArea, reverse=True)[1]          # 计算最大轮廓的旋转包围盒
    rect = cv2.minAreaRect(c)
    angle = rect[2]
    print("angle",angle)
    box = np.int0(cv2.boxPoints(rect))
    draw_img = cv2.drawContours(original_img.copy(), [box], -1, (0, 0, 255), 3)
#     rows, cols = original_img.shape[:2]
#     M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
#     result_img = cv2.warpAffine(original_img, M, (cols, rows))
#     return result_img,draw_img
    return box,draw_img

def Perspective_transform(box,original_img):
    # 获取画框宽高(x=orignal_W,y=orignal_H)
    orignal_W = math.ceil(np.sqrt((box[3][1] - box[2][1])**2 + (box[3][0] - box[2][0])**2))
    orignal_H= math.ceil(np.sqrt((box[3][1] - box[0][1])**2 + (box[3][0] - box[0][0])**2))

    # 原图中的四个顶点,与变换矩阵
    pts1 = np.float32([box[0], box[1], box[2], box[3]])
    pts2 = np.float32([[int(orignal_W+1),int(orignal_H+1)], [0, int(orignal_H+1)], [0, 0], [int(orignal_W+1), 0]])

    # 生成透视变换矩阵；进行透视变换
    M = cv2.getPerspectiveTransform(pts1, pts2)
    result_img = cv2.warpPerspective(original_img, M, (int(orignal_W+3),int(orignal_H+1)))
    print("获取画框宽高，", orignal_W, orignal_H)
    print("原图大小，", original_img.size, original_img.shape)
    print("图片大小，", result_img.size, result_img.shape)
    return result_img

In [34]:
original_img, gray_img, RedThresh, closed, opened = Img_Outline(file_path)
box,draw_img = findContours_img(original_img,RedThresh)
result_img = Perspective_transform(box,original_img)

# show_img([original_img, gray_img, RedThresh, closed, opened])
show_img([original_img, result_img, draw_img])

angle -3.1622977256774902
获取画框宽高， 250 121
原图大小， 3502080 (912, 1280, 3)
图片大小， 92598 (122, 253, 3)


In [38]:
def fourier_demo():
    #1、灰度化读取文件，
    img = cv2.imread(file_path,0)

    #2、图像延扩
    h, w = img.shape[:2]
    new_h = cv2.getOptimalDFTSize(h)
    new_w = cv2.getOptimalDFTSize(w)
    right = new_w - w
    bottom = new_h - h
    nimg = cv2.copyMakeBorder(img, 0, bottom, 0, right, borderType=cv2.BORDER_CONSTANT, value=0)
#     cv2.imshow('new image', nimg)

    #3、执行傅里叶变换，并过得频域图像
    f = np.fft.fft2(nimg)
    fshift = np.fft.fftshift(f)
    magnitude = np.log(np.abs(fshift))


    #二值化
    magnitude_uint = magnitude.astype(np.uint8)
    ret, thresh = cv2.threshold(magnitude_uint, 11, 255, cv2.THRESH_BINARY)
    print(ret)

#     cv2.imshow('thresh', thresh)
    print(thresh.dtype)
    #霍夫直线变换
    lines = cv2.HoughLinesP(thresh, 2, np.pi/180, 30, minLineLength=40, maxLineGap=100)
    print(len(lines))

    #创建一个新图像，标注直线
    lineimg = np.ones(nimg.shape,dtype=np.uint8)
    lineimg = lineimg * 255

    piThresh = np.pi/180
    pi2 = np.pi/2
    print(piThresh)

    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(lineimg, (x1, y1), (x2, y2), (0, 255, 0), 2)
        if x2 - x1 == 0:
            continue
        else:
            theta = (y2 - y1) / (x2 - x1)
        if abs(theta) < piThresh or abs(theta - pi2) < piThresh:
            continue
        else:
            print(theta)

    angle = math.atan(theta)
    print(angle)
    angle = angle * (180 / np.pi)
    print(angle)
    angle = (angle - 90)/(w/h)
    print(angle)

    center = (w//2, h//2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
#     cv2.imshow('line image', lineimg)
#     cv2.imshow('rotated', rotated)
    return lineimg, rotated

In [40]:
ret1 , ret2 = fourier_demo()
show_img([ret1, ret2])

11.0
uint8
60
0.017453292519943295
-54.0
-57.42857142857143
-63.4
-29.272727272727273
9.6
-31.0
84.0
-53.0
-26.4
-27.8
26.8
-31.2
58.25
0.019801980198019802
19.428571428571427
19.428571428571427
28.666666666666668
52.666666666666664
-28.0
-19.333333333333332
-19.166666666666668
14.166666666666666
18.5
-55.666666666666664
23.8
-28.2
19.6
-19.0
-20.5
14.166666666666666
-0.08633093525179857
-0.08611741357726795
-4.934164340560066
-67.64059209264906
